In [1]:
from bs4 import BeautifulSoup as BS
import pymongo
from splinter import Browser
import requests
import time
import pandas as pd

# Current News from NASA Website

In [2]:
#set up splinter browser
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless = False)

#visit url
url = "https://mars.nasa.gov/news/"
browser.visit(url)

#pull html text and parse
html_code = browser.html
soup = BS(html_code, "html.parser")

#grab needed info
news_title = soup.find('div', class_="bottom_gradient").text
news_p = soup.find('div', class_="rollover_description_inner").text

print(news_title)
print(news_p)

NASA’s First Mission to Study the Interior of Mars Awaits May 5 Launch
All systems are go for NASA’s next launch to the Red Planet. 


# Lastest Featured Image

In [3]:
url ="https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
url_base = 'https://www.jpl.nasa.gov'

In [4]:
response = requests.get(url)
soup = BS(response.text, 'html.parser')

result = soup.find('article', class_='carousel_item').attrs

print(result)

{'alt': 'Translucent Rings', 'class': ['carousel_item'], 'style': "background-image: url('/spaceimages/images/wallpaper/PIA18295-1920x1200.jpg');"}


In [5]:
style = str(result['style'])
trim1 = style.replace("background-image:", "")
trim2 = trim1.replace(" url('", "")
image = trim2.replace("');", "")
featured_image_url = url_base + image
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18295-1920x1200.jpg


# Mars Weather

In [6]:
#url of the twitter account
url = "https://twitter.com/marswxreport?lang=en"

In [7]:
response = requests.get(url)
soup = BS(response.text, 'html.parser')

In [8]:
mars_weather = soup.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
mars_weather

'Sol 2037 (April 30, 2018), Sunny, high -2C/28F, low -75C/-103F, pressure at 7.25 hPa, daylight 05:24-17:20'

# Mars Facts

In [10]:
#mars facts url and splinter visit
facts_url = "https://space-facts.com/mars/"
browser.visit(facts_url)

#get html
facts_html = browser.html

soup = BS(facts_html, 'html.parser')

In [11]:
#get the entire table
table_data = soup.find('table', class_="tablepress tablepress-id-mars")

In [16]:
#find all instances of table row
table_all = table_data.find_all('tr')

#set up lists to hold td elements which alternate between label and value
parameters = []
values = []

#for each tr element append the first td element to labels and the second to values
for tr in table_all:
    td_elements = tr.find_all('td')
    parameters.append(td_elements[0].text)
    values.append(td_elements[1].text)

In [17]:
#make a data frame and view
mars_facts_df = pd.DataFrame({
    "Parameters": parameters,
    "Values": values
})

In [18]:
mars_facts_df

,Parameters,Values
0,Equatorial Diameter:,"6,792 km\n"
1,Polar Diameter:,"6,752 km\n"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)\n
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [22]:
# get html code for DataFrame
fact_table = mars_facts_df.to_html(header = False, index = False)
fact_table

'<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km\\n</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km\\n</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)\\n</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

# Mars Hemisphere

In [19]:
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
url_base = "https://astrogeology.usgs.gov"

browser = Browser('chrome', headless=False)
browser.visit(url)

In [20]:
soup = BS(browser.html, 'html.parser')

results = soup.find_all('div', class_="item")

url_list = []

for result in results:
    link = result.find('a')['href']
    url_list.append(link)
    
print(url_list)

['/search/map/Mars/Viking/cerberus_enhanced', '/search/map/Mars/Viking/schiaparelli_enhanced', '/search/map/Mars/Viking/syrtis_major_enhanced', '/search/map/Mars/Viking/valles_marineris_enhanced']


In [21]:
hemisphere_url_images = []

for x in url_list:
    url = url_base + x
        
    browser.visit(url)
    
    # Sleep script to ensure the page fully loads
    time.sleep(5)
    
    soup = BS(browser.html, 'html.parser')
    
    # Grab image url
    result1 = soup.find('img', class_="wide-image")
    image = url_base + result1["src"]
    
    # Grab page title and remove "Enhanced" from string
    result2 = soup.find('h2', class_='title')
    title = result2.text
    title = title.rsplit(' ', 1)[0]
    
    dict_hem = {"Title": title, "Image URL": image}
    hemisphere_url_images.append(dict_hem)
    
    time.sleep(10)
    
print(hemisphere_url_images)

[{'Title': 'Cerberus Hemisphere', 'Image URL': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'Title': 'Schiaparelli Hemisphere', 'Image URL': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'Title': 'Syrtis Major Hemisphere', 'Image URL': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'Title': 'Valles Marineris Hemisphere', 'Image URL': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]
